# Designing and Analyzing the Results of an A/B Test

## Experiment Overview

Udacity is an online education company.  In this experiment, they tested a change to their course sign-up.  After clicking the “Start free trial” button, users were prompted to state how many hours per week they could dedicate to the course.  If a user entered fewer than five hours then they were told that they would need to devote more time than that.  In addition, Udacity proffered the information that the course materials could be accessed for free if the user was unsure of their commitment.  If the user entered more than five hours then they were directed to enter their credit card information and start their 14-day free trial.

<img src = "course-sign-up.png">
> Screen capture of the experimental treatment

In creating this change, Udacity was interested in determining if this “warning” would help users to better understand the commitment required of the course, and hence decrease future frustration.  Udacity hopes this will lead more users to finish their free trial and continue to remain enrolled into the first pay period.

## Experiment Design

### Metric Choices

Invariant Metrics:
* Number of unique cookies to view the course overview page
* Number of unique clicks to click the “Start free trial” button
* Click-through-probability of clicking “Start free trial” button from the course overview page

Evaluation Metrics:
* Gross conversion: number of user-ids to complete checkout divided by the number of unique cookies to click the “Start free trial” button
* Net conversion: number of user-ids to remain enrolled past the 14-day boundary divided by the number of unique cookies to click the “Start free trial” button


#### Invariant Metrics

Invariant metrics shouldn’t change across the experiment and control groups.  We use invariant metrics to size our experiment and ensure that our experiment is run properly.  

I chose the following as invariant metrics:
* Number of unique cookies to view the course overview page
* Number of unique clicks to click the “Start free trial” button
* Click-through-probability of clicking “Start free trial” button from the course overview page

Since we are not making any changes before the user presses the button, we would want the same number of clicks to the button and the same click-through-probability for it, as well as the same number of cookies (as a proxy for users) to visit the course overview page in both the experiment and control groups.  A metric I would not use for invariant checking is the number of users who enrolled in the free trial.  Our experiment may, and is meant to, affect how many users decide to enroll by possibly convincing the less fervent to access free course materials in lieu of starting the free trial.

These metrics were chosen as a way to make sure our two groups are sized similarly and to provide sanity checks.  The experiment does not affect any part of the user experience before the pressing of the “Start free trial” button, so if there are differences between the experimental and control groups for these metrics there is an error with the setup of the experiment, event-capturing, etc.

#### Evaluation Metrics

Evaluation metrics are used to compare the experiment and control groups after the treatment has been applied; these metrics are used to determine if the experiment was a success.

I chose the following as evaluation metrics:
* Gross conversion: number of user-ids to complete checkout divided by the number of unique cookies to click the “Start free trial” button
* Net conversion: number of user-ids to remain enrolled past the 14-day boundary divided by the number of unique cookies to click the “Start free trial” button

To understand the effects of the experimental treatment we need to use two evaluation metrics: gross conversion and net conversion.  Gross conversion quantifies how the experimental change affects the number of users that complete the checkout after clicking on the button.  This helps Udacity understand what portion of their users are abandoning the free trial sign-up in favor of accessing the free course materials due to the experimental treatment.  

Net conversion is also a useful evaluation metric to track since it communicates the overall effect of “warning” users from clicking the free trial button to entering the first pay period.  If the portion of users increases for the first metric, but overall there is not enough of an increase, the change may not be worth launching.

### Measuring Standard Deviation

In [11]:
import pandas as pd
import math

In [12]:
xlsx = pd.ExcelFile('baseline_values.xlsx')
baseline = pd.read_excel(xlsx, 'Sheet1', header=None, names = ['full_metric_name','value'])
baseline.set_index([['pageviews', 'clicks', 'enrollments', 'ctp', 'p_enroll_click', 'p_pay_enroll', 'p_pay_click']], inplace=True)
baseline

,full_metric_name,value
pageviews,Unique cookies to view course overview page pe...,40000.000000
clicks,"Unique cookies to click ""Start free trial"" per...",3200.000000
enrollments,Enrollments per day:,660.000000
ctp,"Click-through-probability on ""Start free trial"":",0.080000
p_enroll_click,"Probability of enrolling, given click:",0.206250
p_pay_enroll,"Probability of payment, given enroll:",0.530000
p_pay_click,"Probability of payment, given click",0.109313


In [14]:
scale_factor = 5000/baseline.loc['pageviews', 'value']
scale_factor

0.125

In [15]:
values = baseline['value'].tolist()

# Scale pageviews, clicks, enrollments for the sample of 5000 pageviews
# Leave the probabilities the same
sample_vals = []
for i in range(0, len(values)):
    if i < 3:
        sample_vals.append(values[i]*scale_factor)
    else:
        sample_vals.append(values[i])
        
baseline['sample'] = sample_vals
baseline

,full_metric_name,value,sample
pageviews,Unique cookies to view course overview page pe...,40000.000000,5000.000000
clicks,"Unique cookies to click ""Start free trial"" per...",3200.000000,400.000000
enrollments,Enrollments per day:,660.000000,82.500000
ctp,"Click-through-probability on ""Start free trial"":",0.080000,0.080000
p_enroll_click,"Probability of enrolling, given click:",0.206250,0.206250
p_pay_enroll,"Probability of payment, given enroll:",0.530000,0.530000
p_pay_click,"Probability of payment, given click",0.109313,0.109313


#### Gross Conversion

In [20]:
# p = probability of enrolling given click
p = baseline.loc['p_enroll_click', 'value']
q = 1 - p

gc_std = math.sqrt(p*q/baseline.loc['clicks', 'sample'])
gc_std

0.020230604137049392

#### Net Conversion

In [21]:
# p = probability of making a payment given click
p = baseline.loc['p_pay_click', 'value']
q = 1 - p

nc_std = math.sqrt(p*q/baseline.loc['clicks', 'sample'])
nc_std

0.01560154458248846

### Sizing

## Experiment Analysis

### Sanity Checks